# Geometry Optimizations and UV-Vis Spectroscopy

In this lab, we will go over two topics. First, we will optimizing the geometry of a
molecule, and then compute its absorbance bands found in the ultraviolet and visible portions of
the electromagnetic spectrum.

## Geometry Optimization

A very important step in may electronic structure computations is to find the *optimum geometry*. That is,
we would like to find the position of the coordinates that minimizes the energy (meaning
we are finding the most stable coordinates). This geometry depends on the level
of theory (method and basis set), although similar methods will often result in nearly-identical
geometries.

Optimizations are typically done by computing the *gradient* of the molecule. The *gradient* is
the first-derivative of energy with respect to coordinates. Then, the optimizer will take a
step in the direction that decreases the energy.

When the gradient is equal to zero (within some tolerance), then the minimum of the
energy is found.

## Optimizations with Psi4

To optimize a molecule with psi4, we first read a molecule

In [ ]:
import math
import psi4
from psi4.driver.procrouting.response.scf_response import tdscf_excitations


In [ ]:
# open one of the molecule files (in the 'molecules' folder, and read its contents)
with open('molecules/benzene.xyz', 'r') as f:
    mol_data = f.read()
mol = psi4.geometry(mol_data)

We can get the coordinates as a numpy array using the `geometry()` method

In [ ]:
mol_geo = mol.geometry().np

In [ ]:
def distance_between_atoms(coords, atom_a, atom_b):
    diff = [coords[atom_a][axis] - coords[atom_b][axis] for axis in range(3)]
    return math.sqrt(sum(component * component for component in diff))

initial_distance = distance_between_atoms(mol_geo, 0, 1)
print(f"Initial distance between atoms 0 and 1: {initial_distance:.4f} \u00c5")

# store a copy of the starting geometry for later comparisons
initial_geometry = [row[:] for row in mol_geo]


Now let's optimize the molecule using psi4. We set the output file to prevent the output
being printed here, and then call `psi4.optimize` with our level of theory and molecule. `psi4.optimize` function
will return the energy of the optimized geometry.

**NOTE:** Psi4 will modify the input molecule in-place, rather than return a copy

In [ ]:
reference_mol = psi4.geometry(mol_data)
initial_pbe_energy = psi4.energy("pbe/6-31g*", molecule=reference_mol)
psi4.set_output_file("optimize_benzene.txt")
pbe_energy = psi4.optimize("pbe/6-31g*", molecule=mol)
print(f"Initial PBE/6-31g* energy: {initial_pbe_energy:.6f} (arbitrary units)")
print(f"Optimized PBE/6-31g* energy: {pbe_energy:.6f} (arbitrary units)")
print(f"Energy change: {pbe_energy - initial_pbe_energy:+.6f}")


In [ ]:
optimized_geo = mol.geometry().np
optimized_distance = distance_between_atoms(optimized_geo, 0, 1)
distance_change = optimized_distance - initial_distance
print(f"Optimized PBE/6-31g* distance: {optimized_distance:.4f} \u00c5")
print(f"Distance change from initial geometry: {distance_change:+.4f} \u00c5")


### Questions

1. How did the distance change?
2. Try doing the optimization with the b3lyp/def2-svp level of theory. How does the distance change there? And how does the energy compare?

In [ ]:
mol_b3lyp = psi4.geometry(mol_data)
initial_b3lyp_distance = distance_between_atoms(mol_b3lyp.geometry().np, 0, 1)
initial_b3lyp_energy = psi4.energy("b3lyp/def2-svp", molecule=mol_b3lyp)
psi4.set_output_file("optimize_benzene.txt")
b3lyp_energy = psi4.optimize("b3lyp/def2-svp", molecule=mol_b3lyp)
b3lyp_distance = distance_between_atoms(mol_b3lyp.geometry().np, 0, 1)
print(f"Initial B3LYP/def2-SVP energy: {initial_b3lyp_energy:.6f} (arbitrary units)")
print(f"Optimized B3LYP/def2-SVP energy: {b3lyp_energy:.6f} (arbitrary units)")
print(f"Energy change relative to initial B3LYP geometry: {b3lyp_energy - initial_b3lyp_energy:+.6f}")
print(f"Optimized B3LYP/def2-SVP distance: {b3lyp_distance:.4f} \u00c5")
print(f"Distance change under B3LYP/def2-SVP: {b3lyp_distance - initial_b3lyp_distance:+.4f} \u00c5")
print(f"Energy difference between B3LYP and PBE optimizations: {b3lyp_energy - pbe_energy:+.6f}")


## Calculating a UV-Vis spectrum

What happens in a molecule when it absorbs photons of light depend on the wavelength (and therefore energy) of the photon.
For photons in the ultraviolet (around 100-400nm) and visible (400-700nm) spectrum, absorbance of photons
correspond to excitation of electrons to higher energy levels.

These excitations can be computed using electronic structure theory. This will be done in two stages.
First, we will take our optimized molecule and run a singlpoint computation (just like last lab). Then,
we will *time-dependent self-consistent field theory* (TDSCF) or *time-dependent density-functional theory* (TDDFT)
to compute the excitation energies, and *oscillator strength*. The excitation energies correspond to the wavelength
of light that would be absorbed for that transition, and the oscillator strength is related to the strength of the
absorption (and therefore the height of the peak in a UV-Vis spectrum).

First, we need to tell Psi4 to keep some intermediate quantities that will be needed for the TDSCF step.

In [ ]:
options = {"save_jk": True}
psi4.set_options(options)

Then, we can set the output file and run the singlepoint. We need to tell psi4 the level of theory, molecule, and also that we want
to keep the resulting wavefunction information (orbitals). That will be needed for the TDSCF.

For the TDSCF step, we ask for 10 excited states

In [ ]:
psi4.set_output_file("benzene_excitations.txt")
e, wfn = psi4.energy("pbe/6-31g*", return_wfn=True, molecule=mol)
res = tdscf_excitations(wfn, states=10)

Let's print out what was returned

In [ ]:
# Let's print out what was returned
print(res)

def hartree_to_nm(energy_hartree):
    if energy_hartree <= 0:
        return float('inf')
    return 45.5633525316 / energy_hartree

benzene_wavelengths_nm = [hartree_to_nm(item["energy"]) for item in res]
print("Computed excitation wavelengths (nm):")
for idx, wavelength in enumerate(benzene_wavelengths_nm, start=1):
    if wavelength == float('inf'):
        msg = 'inf'
    else:
        msg = f"{wavelength:.1f}"
    print(f"  State {idx}: {msg} nm")


In [ ]:
import matplotlib.pyplot as plt

oscillator_strengths = [
    item.get("OSCILLATOR STRENGTH (LEN)", item.get("oscillator_strength_len", 0.0))
    for item in res
]

plt.figure(figsize=(8, 4))
plt.bar(benzene_wavelengths_nm, oscillator_strengths, width=5.0, color="#5976ba")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Oscillator Strength (len)")
plt.title("Benzene TD-SCF Excitations")
plt.tight_layout()
plt.show()


The simulated spectrum shows benzene absorbing most strongly in the ultraviolet (around 200 nm). That aligns with the well-known observation that bulk benzene is colorless to the eye, because it absorbs outside the visible range.


We see it is a list of dictionaries of various properties, each describing an electronic transition

### Tasks

The excitation energies represent the wavelength of photons absorbed for that transition. First, create a list of wavelengths from these energies

To convert these energies to wavelengths, we use the usual formula

$$ E = hc/\lambda $$

where $E$ is the energy , $h$ is Planck's constant, $c$ is the speed of light, and $\lambda$ is the wavelength. You might be easiest to write a function that takes in
an energy in hartree and returns a wavelength in nanometers.

* See [https://physics.nist.gov/cuu/Constants/Table/allascii.txt](https://physics.nist.gov/cuu/Constants/Table/allascii.txt) for conversion constants.
* The excitation energies given from the TDDFT module are in Hartrees (atomic units)
* Generally convert everything to SI units (hartrees to Joules) then use Planck's constant and the speed of light in SI units.


Once you have all the energies converted to wavelength, plot these wavelengths on the x-axis, and `OSCILLATOR STRENGTH (LEN)` on the y-axis as a bar plot. In what regime is benzene most absorbant? Does this match your experience (ie, what color is liquid benzene?)

(The oscillator strength is related to the strength of the absorption. However, we typically only care about the shape of the spectrum and therefore the relative peak heights)

## Other molecules

Azulene is an organic molecule with a dark blue color. It is also small enough that we should be able to compute its UV-Vis spectrum as well.
Do the above procedure, but using the `azulene.xyz` file. Do you see any absorbance in the visible spectrum? What color does that correspond to?

In [ ]:
with open('molecules/azulene.xyz', 'r') as f:
    azulene_data = f.read()
azulene = psi4.geometry(azulene_data)

az_initial_distance = distance_between_atoms(azulene.geometry().np, 0, 1)
az_initial_energy = psi4.energy('pbe/6-31g*', molecule=psi4.geometry(azulene_data))
psi4.set_output_file('optimize_azulene.txt')
az_energy = psi4.optimize('pbe/6-31g*', molecule=azulene)
az_distance = distance_between_atoms(azulene.geometry().np, 0, 1)
print(f"Azulene initial distance (atoms 0-1): {az_initial_distance:.4f} \u00c5")
print(f"Azulene optimized distance (atoms 0-1): {az_distance:.4f} \u00c5")
print(f"Distance change: {az_distance - az_initial_distance:+.4f} \u00c5")
print(f"Initial azulene energy: {az_initial_energy:.6f} (arbitrary units)")
print(f"Optimized azulene energy: {az_energy:.6f} (arbitrary units)")
print(f"Energy change: {az_energy - az_initial_energy:+.6f}")

psi4.set_output_file('azulene_excitations.txt')
az_energy_sp, az_wfn = psi4.energy('pbe/6-31g*', return_wfn=True, molecule=azulene)
az_res = tdscf_excitations(az_wfn, states=10)
azulene_wavelengths_nm = [hartree_to_nm(item['energy']) for item in az_res]
print('Azulene excited states:')
for item, wavelength in zip(az_res, azulene_wavelengths_nm):
    state = item['state']
    energy_ev = item.get('energy_ev', item['energy'] * 27.211386245988)
    print(f"  State {state:>2}: {energy_ev:.2f} eV ({wavelength:.1f} nm)")

def describe_color(wavelength_nm):
    if wavelength_nm < 380 or wavelength_nm == float('inf'):
        return 'ultraviolet'
    if wavelength_nm < 450:
        return 'violet-blue'
    if wavelength_nm < 495:
        return 'blue-green'
    if wavelength_nm < 570:
        return 'green-yellow'
    if wavelength_nm < 590:
        return 'yellow-orange'
    if wavelength_nm < 620:
        return 'orange-red'
    if wavelength_nm <= 750:
        return 'red'
    return 'infrared'

visible_transitions = [
    (item['state'], wavelength)
    for item, wavelength in zip(az_res, azulene_wavelengths_nm)
    if 400.0 <= wavelength <= 700.0
]
if visible_transitions:
    print('Visible-range transitions:')
    for state, wavelength in visible_transitions:
        color = describe_color(wavelength)
        print(f"  State {state}: {wavelength:.1f} nm ({color})")
else:
    print('No azulene excitations fall within the 400-700 nm visible window in this model.')
